# Import and preprocess of italina Covi-19 dataset

In [1]:
import pandas as pd
import glob
import os

## Goal:

Create regional and national dataset whit information about covid-19 epidemic

## Dataset structure

Structure of dataset are same for national and regional scale, below the explanation of columns: <br>
<ul>
<li><b>total_cases: </b>cumulative number of infect<br></li>
<li><b>daily_cases: </b>daily new infect<br></li>
<li><b>actual_cases: </b>people actually infect<br></li>
<li><b>total_healed: </b>cumulative number of healed<br></li>
<li><b>daily_healed: </b>daily number of healed<br></li>
<li><b>total_dead: </b>cumulative number of dead<br></li>
<li><b>daily_dead: </b>daily number of dead<br></li>
<li><b>total_home_isolation: </b>cumulative number of people forced to home isolation<br></li>
<li><b>daily_home_isolation: </b>daily number of people forced to home isolation, variation between day before, can be negative<br></li>
<li><b>total_recovered_with_symptoms: </b>cumulative number of people recovered with symptoms<br></li>
<li><b>daily_recovered_with_symptoms: </b>daily number of people recovered with symptoms, variation between day before, can be negative<br></li>
<li><b>total_intensive_care: </b>cumulative number of people in intesive care<br></li>
<li><b>daily_intensive_care: </b>daily number of people in intesive care, variation between day before, can be negative<br></li>
<li><b>total_hospitalized: </b>cumulative number of hospitalized: simple recover + intensive care<br></li>
<li><b>daily_hospitalized: </b>daily number of hospitalized: simple recover + intensive care, variation between day before, can be negative<br></li>
<li><b>total_test: </b>cumulative number of test made<br></li>
<li><b>daily_test: </b>daily number of test made<br></li>
</ul>

In [2]:
#define colums of final dataset
dtregions_cols = ["total_cases","daily_cases","actual_cases","total_healed","daily_healed","total_dead","daily_dead",
                  "total_home_isolation","daily_home_isolation","total_recovered_with_symptoms","daily_recovered_with_symptoms",
                  "total_intensive_care","daily_intensive_care","total_hospitalized","daily_hospitalized","total_test","daily_test"]

#use rigth line, depens on your OS !!!
#path = os.getcwd() + "/Data/Italia/COVID-19-master/dati-regioni/"
path = os.getcwd() + "\\Data\\Italia\\COVID-19-master\\dati-regioni\\"

dailycsv = glob.glob(path + "/*.csv")

#dailycsv.remove(os.getcwd() + "/Data/Italia/COVID-19-master/dati-regioni/" + "dpc-covid19-ita-regioni.csv")
#dailycsv.remove(os.getcwd() + "/Data/Italia/COVID-19-master/dati-regioni/" + "dpc-covid19-ita-regioni-latest.csv")
dailycsv.remove(os.getcwd() + "\\Data\\Italia\\COVID-19-master\\dati-regioni\\" + "dpc-covid19-ita-regioni.csv")
dailycsv.remove(os.getcwd() + "\\Data\\Italia\\COVID-19-master\\dati-regioni\\" + "dpc-covid19-ita-regioni-latest.csv")

#define container for dataset
dtregions_ita_array = {} 



In [3]:
#read all daily CSV then put into dataset divided by region
for file in dailycsv:
    dtregions_ita = pd.read_csv(file, index_col = 0)
    for region in dtregions_ita["denominazione_regione"].values:
        try:
            dtregions_ita_array[region] = dtregions_ita_array[region].append(dtregions_ita.loc[dtregions_ita["denominazione_regione"] == region])
        except:
            dtregions_ita_array[region] = dtregions_ita.loc[dtregions_ita["denominazione_regione"] == region]
        dtregions_ita_array[region].drop(['stato', 'codice_regione', 'denominazione_regione', 'lat', 'long'], axis = 1, inplace = True)

D:\Programmi\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
D:\Programmi\Anaconda\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
#translate some name and sort dataset by date
for region in dtregions_ita_array:
    dtregions_ita_array[region].index.name = "date"
    dtregions_ita_array[region].sort_values(by=['date'], inplace = True)
    dtregions_ita_array[region].columns=['dead','healed', 'home_isolation',
       'new_actual_positive', 'hospitalized_with_symptoms', 'test',
       'intensive_care', 'total_actual_positive', 'total_cases',
       'total_hospitalized']  

In [5]:
#create sample datatset
dt_sample = dtregions_ita_array["Lombardia"].copy()[[]]
dt_sample = dt_sample.reindex(columns=dt_sample.columns.tolist() + dtregions_cols)

In [6]:
#define containmer for dataset
dtregions_array = {} 

#create one dataset forach region and start filling with avaible data
for region in dtregions_ita_array:
    dtregions_array[region] = dt_sample.copy()
    dtregions_array[region].total_cases = dtregions_ita_array[region].total_cases
    dtregions_array[region].total_healed = dtregions_ita_array[region].healed
    dtregions_array[region].total_dead = dtregions_ita_array[region].dead
    dtregions_array[region].total_recovered_with_symptoms = dtregions_ita_array[region].hospitalized_with_symptoms
    dtregions_array[region].total_intensive_care = dtregions_ita_array[region].intensive_care
    dtregions_array[region].total_home_isolation = dtregions_ita_array[region].home_isolation
    dtregions_array[region].total_hospitalized = dtregions_ita_array[region].total_hospitalized
    dtregions_array[region].total_test = dtregions_ita_array[region].test

In [7]:
#calculate the other data 
for region in dtregions_array:
    for i in range(len(dtregions_array[region].index)):
        if i == 0:
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_cases")] = dtregions_array[region].iloc[i].total_cases
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_healed")] = dtregions_array[region].iloc[i].total_healed
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_dead")] = dtregions_array[region].iloc[i].total_dead
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("actual_cases")] = dtregions_array[region].iloc[i].total_cases - dtregions_array[region].iloc[i].total_dead- dtregions_array[region].iloc[i].total_healed
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_home_isolation")] = dtregions_array[region].iloc[i].total_home_isolation
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_recovered_with_symptoms")] = dtregions_array[region].iloc[i].total_recovered_with_symptoms
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_intensive_care")] = dtregions_array[region].iloc[i].total_intensive_care
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_hospitalized")] = dtregions_array[region].iloc[i].total_hospitalized
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_test")] = dtregions_array[region].iloc[i].total_test
        else:
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_cases")] = dtregions_array[region].iloc[i].total_cases - dtregions_array[region].iloc[i-1].total_cases
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_dead")] = dtregions_array[region].iloc[i].total_dead - dtregions_array[region].iloc[i-1].total_dead
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_healed")] = dtregions_array[region].iloc[i].total_healed - dtregions_array[region].iloc[i-1].total_healed
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("actual_cases")] = dtregions_array[region].iloc[i].total_cases - dtregions_array[region].iloc[i].total_dead- dtregions_array[region].iloc[i].total_healed
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_home_isolation")] = dtregions_array[region].iloc[i].total_home_isolation - dtregions_array[region].iloc[i-1].total_home_isolation
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_recovered_with_symptoms")] = dtregions_array[region].iloc[i].total_recovered_with_symptoms - dtregions_array[region].iloc[i-1].total_recovered_with_symptoms
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_intensive_care")] = dtregions_array[region].iloc[i].total_intensive_care - dtregions_array[region].iloc[i-1].total_intensive_care
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_hospitalized")] = dtregions_array[region].iloc[i].total_hospitalized - dtregions_array[region].iloc[i-1].total_hospitalized
            dtregions_array[region].iloc[i, dtregions_array[region].columns.get_loc("daily_test")] = dtregions_array[region].iloc[i].total_test - dtregions_array[region].iloc[i-1].total_test    

Creiamo adesso un Dataframe raggruppando tutte le informazioni di ogni singola regione per singola data

In [8]:
dtregions_array["Lombardia"].head()

,total_cases,daily_cases,actual_cases,total_healed,daily_healed,total_dead,daily_dead,total_home_isolation,daily_home_isolation,total_recovered_with_symptoms,daily_recovered_with_symptoms,total_intensive_care,daily_intensive_care,total_hospitalized,daily_hospitalized,total_test,daily_test
date,,,,,,,,,,,,,,,,,
2020-02-24 18:00:00,172,172.0,166.0,0,0.0,6,6.0,71,71.0,76,76.0,19,19.0,95,95.0,1463,1463.0
2020-02-25 18:00:00,240,68.0,231.0,0,0.0,9,3.0,127,56.0,79,3.0,25,6.0,104,9.0,3700,2237.0
2020-02-26 18:00:00,258,18.0,249.0,0,0.0,9,0.0,145,18.0,79,0.0,25,0.0,104,0.0,3208,-492.0
2020-02-27 18:00:00,403,145.0,349.0,40,40.0,14,5.0,136,-9.0,172,93.0,41,16.0,213,109.0,3320,112.0
2020-02-28 18:00:00,531,128.0,474.0,40,0.0,17,3.0,192,56.0,235,63.0,47,6.0,282,69.0,4835,1515.0


In [9]:
dtregions_array["Lombardia"].tail()

,total_cases,daily_cases,actual_cases,total_healed,daily_healed,total_dead,daily_dead,total_home_isolation,daily_home_isolation,total_recovered_with_symptoms,daily_recovered_with_symptoms,total_intensive_care,daily_intensive_care,total_hospitalized,daily_hospitalized,total_test,daily_test
date,,,,,,,,,,,,,,,,,
2020-03-20 17:00:00,22264,2380.0,15420.0,4295,517.0,2549,381.0,6635,1090.0,7735,348.0,1050,44.0,8785,392.0,57174,4930.0
2020-03-21 17:00:00,25515,3251.0,17370.0,5050,755.0,3095,546.0,8019,1384.0,8258,523.0,1093,43.0,9351,566.0,66730,9556.0
2020-03-22 17:00:00,27206,1691.0,17885.0,5865,815.0,3456,361.0,7304,-715.0,9439,1181.0,1142,49.0,10581,1230.0,70598,3868.0
2020-03-23 17:00:00,28761,1555.0,18910.0,6075,210.0,3776,320.0,8461,1157.0,9266,-173.0,1183,41.0,10449,-132.0,73242,2644.0
2020-03-24 17:00:00,30703,1942.0,19868.0,6657,582.0,4178,402.0,8963,502.0,9711,445.0,1194,11.0,10905,456.0,76695,3453.0


In [34]:
dtitaly = dt_sample.copy()
dtitaly.fillna(0, inplace=True)

for i in range(len(dtregions_array["Lombardia"])):
    for region in dtregions_array:
        dtitaly.iloc[i] += dtregions_array[region].iloc[i]

In [35]:
dtitaly

,total_cases,daily_cases,actual_cases,total_healed,daily_healed,total_dead,daily_dead,total_home_isolation,daily_home_isolation,total_recovered_with_symptoms,daily_recovered_with_symptoms,total_intensive_care,daily_intensive_care,total_hospitalized,daily_hospitalized,total_test,daily_test
date,,,,,,,,,,,,,,,,,
2020-02-24 18:00:00,229.0,229.0,221.0,1.0,1.0,7.0,7.0,94.0,94.0,101.0,101.0,26.0,26.0,127.0,127.0,4324.0,4324.0
2020-02-25 18:00:00,322.0,93.0,311.0,1.0,0.0,10.0,3.0,162.0,68.0,114.0,13.0,35.0,9.0,150.0,23.0,8623.0,4299.0
2020-02-26 18:00:00,400.0,78.0,385.0,3.0,2.0,12.0,2.0,221.0,59.0,128.0,14.0,36.0,1.0,164.0,14.0,9587.0,964.0
2020-02-27 18:00:00,650.0,250.0,588.0,45.0,42.0,17.0,5.0,284.0,63.0,248.0,120.0,56.0,20.0,304.0,140.0,12014.0,2427.0
2020-02-28 18:00:00,888.0,238.0,821.0,46.0,1.0,21.0,4.0,412.0,128.0,345.0,97.0,64.0,8.0,409.0,105.0,15695.0,3681.0
2020-02-29 17:00:00,1128.0,240.0,1049.0,50.0,4.0,29.0,8.0,543.0,131.0,401.0,56.0,105.0,41.0,506.0,97.0,18661.0,2966.0
2020-03-01 17:00:00,1694.0,566.0,1577.0,83.0,33.0,34.0,5.0,798.0,255.0,639.0,238.0,140.0,35.0,779.0,273.0,21127.0,2466.0
2020-03-02 18:00:00,2036.0,342.0,1835.0,149.0,66.0,52.0,18.0,927.0,129.0,742.0,103.0,166.0,26.0,908.0,129.0,23345.0,2218.0
2020-03-03 18:00:00,2502.0,466.0,2263.0,160.0,11.0,79.0,27.0,1000.0,73.0,1034.0,292.0,229.0,63.0,1263.0,355.0,25856.0,2511.0
